##### DNN CNN
    - 독립적인 정보
    - 입력(x) 간의 순서나 연관성을 고려하지 않는다
##### 시계열 : 시간의 연속적인 흐름
    - 시계열 데이터 : 날씨, 주식, 문장 --> 순서가 중요한 데이터
##### RNN 
    - 순환하는 구조
        - 시점1 (월요일) : 맑음(x1)  정보가 RNN에 들어온다 -> RNN 날씨가 맑았음(h1) 이라는 요약본을 생성
        - 시점2 (화요일) : 흐림(x2)  ->RNN 새로운정보(흐림, x2) + 어제의기억(맑았음, h1) 함께 고려
            어제 맑았는데 오늘 흐림 h2 이라는 새로운 요약본을 생성
        - 시점3 (수요일) : 비(x3) ->RNN 새로운정보(비, x3) 정보와 + 어제의기억(어제 맑았는데 오늘 흐림, h2) 새로운 상태 h3
        - 반복 
    - 알고리즘
        - 각 시점(time step)에서  1. 현재의 입력 과 2. 과거의 기억(hidden state ht-1) 받아서 3. 현재의 결과물과 4. 다음 시점으로 넘겨줄 최신 기억 ht 을 생성
        - ht 기억이 시계열 데이터의 맥락(Context) 저장하는 역할
    - 장점
        - 순서가 있는 데이터의 맥락을 학습
    - 한계
        - 기억력이 생각보다 짧다
        - 시계열 데이터가 길어지면(예 100단계 전의 정보)
        - 이전정보가 소실되거나 반대로 너무 강해져서 폭주가 되서 제대로 학습이 안된다
        - 장기 기억 의존성 문제 (Long-Term Dependency Problem)
##### LSTM & GRU
    - LSTM(Long Short Term Memory) : RNN 내부에 게이트(Gate) 복잡한 장치 -> 잊고, 기억할 정보를 관리
    - GRU(Gated Recurrent Unit) : LSTM 구조를 좀 더 단순화시킨 모델, LSTM성능은 비슷, 속도는 빠르다
##### RNN 핵심수식
    - 은닉상태계산
        - h1 = tanh(wht-1 + wxt - bh)
    - 출력계산

In [ ]:
import pandas as pd
url = 'https://raw.githubusercontent.com/pia222sk20/python/refs/heads/main/data/time_data_train.csv'
df = pd.read_csv(url)
df.head()

In [ ]:
# 전체 데이터셋 확인
df.info()

In [ ]:
# EDA - 탐색적 데이터 분석

In [ ]:
# 데이터셋 정의
import pandas as pd
import numpy as np
from torch.utils.data.dataset import Dataset
from sklearn.preprocessing import StandardScaler
import torch

class StockDataSet(Dataset):
    def __init__(self):
        url = 'https://raw.githubusercontent.com/pia222sk20/python/refs/heads/main/data/time_data_train.csv'
        self.csv = pd.read_csv(url)
        data = torch.Tensor(self.csv.iloc[: , 1: -1].values)
        label = torch.Tensor(self.csv.iloc[: , -1].values.reshape(-1, 1))
        self.data = StandardScaler().fit_transform(data)
        # 정답이 숫자 크다면 정규화가 학습에 도움이 된다.
        self.label = StandardScaler().fit_transform(label)
        self.data = torch.Tensor(self.data)
        self.label = torch.Tensor(self.label)
    def __len__(self):
        return len(self.data) - 30 # 사용가능한 배치 개수
    def __getitem__(self, index):
        data = self.data[index: index + 30]
        label = self.label[index + 30]
        return data, label

In [ ]:
data, label = next(iter(StockDataSet()))
data.size(), label.size()

In [ ]:
from torch import nn
class StockRNN(nn.Module):
    def __init__(self):
        super(StockRNN, self).__init__()
        # (30일, 배치 16개, 각 입력의 특성 4개) batch_first = False
        # (배치 16개, 30일 각 입력의 특성 4개) batch_first = True
        self.rnn = nn.RNN(input_size=4, hidden_size=8, num_layers=5, batch_first=True)
        # 출력 (batch, 30, 8)
        self.fc1 = nn.Linear(30*8, 64)
        self.fc2 = nn.Linear(64, 1)
        self.relu = nn.ReLU()
    def forward(self, x, ho): # 입력데이터는 (16,30,4)
        # ho = 초기 은닉 상태(num_layers, batch, hidden_size) (5,16,8)
        # 출력 x 는 모든 시점에 대한 hidden output을 담고 있어야함 (batch, seq_len, hidden_size) (16,30,8)
        # 출력 hn 최종은닉상태(각 레이어의 마지막 타임스탬프 hidden state) (num_layer, batch, hidden_state) (5,16,8)
        x, hn = self.rnn(x, ho)
        # mlp 입력으로 사용될 수 있도록 모양 변경
        x = torch.reshape(x, (x.shape[0], -1))
        # mlp 
        x = self.relu(self.fc1(x))
        x = self.fc2(x)
        # 예측한 종가 1차원 벡터
        out = torch.flatten(x)
        return out


In [ ]:
rnn = StockRNN()
sample_data = torch.randn(16, 30, 4)
# 초기 hidden state 값  (num_layer, batch_size, hidden_size) (5, 16, 8)
ho = torch.zeros(5,16,8)
out = rnn(sample_data, ho)
out

In [ ]:
from torch.optim.adam import Adam
from torch.utils.data.dataloader import DataLoader
device = 'cuda' if torch.cuda.is_available() else 'cpu'
model = StockRNN().to(device)
dataset = StockDataSet()
loader = DataLoader(dataset, batch_size=16)
optim = Adam(model.parameters(), lr=1e-3)

In [ ]:
# Expected hidden size (5,9,8) got [5,16,8]
# Rnn이 처리하는 배치크기는 9, 우리가 설계한 ho 16
print(len(dataset) % 16)
# 마지막 배치 개수가 모자라서 9

In [ ]:
from tqdm import tqdm
for epoch in range(200):
    loop = tqdm(loader)
    for data, label in loop:
        optim.zero_grad() # 학습 다 끝냈는데 이전 기울기가 남아있는 상황 방지
        # 초기 은닉 상태의 배치크기는 DataLoader가 주는 배치 크기
        batch_size = data.size(0)

        ho = torch.zeros(5,batch_size,8).to(device)
        # 모델의 예측값
        pred = model(data.to(device), ho)
        # 손실값
        loss = nn.MSELoss()(pred, label.to(device))
        loss.backward()
        optim.step()

        loop.set_description(f'epoch: {epoch+1} loss: {loss.item()}')
torch.save(model.state_dict(), './rnn.pth')


In [ ]:
# 모델 성능평가하기
import matplotlib.pyplot as plt

loader = DataLoader(dataset,batch_size=1)
preds = []
total_loss = 0
with torch.no_grad():
    model.load_state_dict(torch.load('rnn.pth', map_location=device,weights_only=False))
    for data, label in loader:
        # 초기 은닉상태의 배치크기는 DataLoader가 주는 배치 크기
        batch_size = data.size(0)

        ho = torch.zeros(5,batch_size,8).to(device)
        # 모델의 예측값
        pred = model(data.to(device), ho)
        preds.append(pred.item())
        # 손실값
        loss = nn.MSELoss()(pred, label.to(device))
        total_loss += (loss.item() / len(loader))

print(f'total_loss : {total_loss:.4f}')

In [ ]:
plt.figure(figsize=(12,5))
plt.plot(preds, label='preds')
plt.plot(dataset.label, label='real')
plt.legend()
plt.show()

In [ ]:
with torch.no_grad():
    last_30 = df.iloc[-30:, 1:-1].values
    X = torch.tensor(last_30, dtype=torch.float32).unsqueeze(0)
    ho = torch.zeros(5,1,8).to(device)
    pred = model(X.to(device), ho)
    print(pred.item())

In [ ]:
# yfinance
X = np.array([
    [10,20],[100,200]
])
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)
X_scaled

In [ ]:
scaler.inverse_transform(X_scaled)

In [ ]:
# %pip install yfinance

In [99]:
import yfinance as yf
dat = yf.Ticker("MSFT")
df = dat.history(period='1y')
df.head()

,Open,High,Low,Close,Volume,Dividends,Stock Splits
Date,,,,,,,
2024-10-29 00:00:00-04:00,424.810714,429.942203,422.627096,428.731293,17644100,0.0,0.0
2024-10-30 00:00:00-04:00,434.180393,435.232492,428.880188,429.306976,29749100,0.0,0.0
2024-10-31 00:00:00-04:00,412.264924,413.058981,403.272437,403.322083,53971000,0.0,0.0
2024-11-01 00:00:00-04:00,405.962252,412.403882,404.463494,407.312103,24230400,0.0,0.0
2024-11-04 00:00:00-05:00,406.746332,407.361738,402.547872,405.416321,19672300,0.0,0.0


In [ ]:
# 독립변수, 종속변수 분리
# window size : 30(한달치 데이터를 하나의 dataset)
# 배치는 3배치
# 알고리즘은 RNN

In [100]:
# 데이터셋 정의
from torch.utils.data.dataset import Dataset
from sklearn.preprocessing import StandardScaler
import torch

class YFieldDataSet(Dataset):
    def __init__(self):
        self.csv = df
        # print(self.csv.iloc[: , : 4].values)
        # print(self.csv.columns[:4])
        data = torch.Tensor(self.csv.iloc[: , :4].values)
        label = torch.Tensor(self.csv.iloc[: , 4].values.reshape(-1, 1))
        self.data = StandardScaler().fit_transform(data)
        # 정답이 숫자 크다면 정규화가 학습에 도움이 된다.
        self.label = StandardScaler().fit_transform(label)
        self.data = torch.Tensor(self.data)
        self.label = torch.Tensor(self.label)
    def __len__(self):
        return len(self.data) - 30 # 사용가능한 배치 개수
    def __getitem__(self, index):
        data = self.data[index: index + 30]
        label = self.label[index + 30]
        return data, label

In [102]:
from torch import nn
class YFieldRNN(nn.Module):
    def __init__(self):
        super(YFieldRNN, self).__init__()
        self.rnn = nn.RNN(input_size=4, hidden_size=8, num_layers=5, batch_first=True)
        self.fc1 = nn.Linear(30*8, 64)
        self.fc2 = nn.Linear(64, 1)
        self.relu = nn.ReLU()
    def forward(self, x, ho): # 입력데이터는 (16,30,4)
        x, hn = self.rnn(x, ho)
        x = torch.reshape(x, (x.shape[0], -1))
        x = self.relu(self.fc1(x))
        x = self.fc2(x)
        out = torch.flatten(x)
        return out


In [103]:
rnn = YFieldRNN()
sample_data = torch.randn(3, 30, 4)
ho = torch.zeros(5,3,8)
out = rnn(sample_data, ho)
out

tensor([-0.0744, -0.0761, -0.0689], grad_fn=<ViewBackward0>)

In [104]:
from torch.optim.adam import Adam
from torch.utils.data.dataloader import DataLoader
device = 'cuda' if torch.cuda.is_available() else 'cpu'
model = YFieldRNN().to(device)
dataset = YFieldDataSet()
loader = DataLoader(dataset, batch_size=3)
optim = Adam(model.parameters(), lr=1e-3)

In [105]:
from tqdm import tqdm
for epoch in range(200):
    loop = tqdm(loader)
    for data, label in loop:
        optim.zero_grad() # 학습 다 끝냈는데 이전 기울기가 남아있는 상황 방지
        batch_size = data.size(0)

        ho = torch.zeros(5,batch_size,8).to(device)
        pred = model(data.to(device), ho)
        loss = nn.MSELoss()(pred, label.to(device))
        loss.backward()
        optim.step()

        loop.set_description(f'epoch: {epoch+1} loss: {loss.item()}')
torch.save(model.state_dict(), './yfinance.pth')


  0%|          | 0/74 [00:00<?, ?it/s]c:\Users\playdata2\miniconda3\envs\deep\Lib\site-packages\torch\nn\modules\loss.py:608: UserWarning: Using a target size (torch.Size([3, 1])) that is different to the input size (torch.Size([3])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)
epoch: 1 loss: 0.343813419342041:  92%|█████████▏| 68/74 [00:01<00:00, 65.57it/s]  c:\Users\playdata2\miniconda3\envs\deep\Lib\site-packages\torch\nn\modules\loss.py:608: UserWarning: Using a target size (torch.Size([1, 1])) that is different to the input size (torch.Size([1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)
epoch: 200 loss: 0.001575491507537663: 100%|██████████| 74/74 [00:01<00:00, 49.10it/s]


In [106]:
# 모델 성능평가하기
import matplotlib.pyplot as plt

loader = DataLoader(dataset,batch_size=1)
preds = []
total_loss = 0
with torch.no_grad():
    model.load_state_dict(torch.load('yfinance.pth', map_location=device,weights_only=False))
    for data, label in loader:
        # 초기 은닉상태의 배치크기는 DataLoader가 주는 배치 크기
        batch_size = data.size(0)

        ho = torch.zeros(5,batch_size,8).to(device)
        # 모델의 예측값
        pred = model(data.to(device), ho)
        preds.append(pred.item())
        # 손실값
        loss = nn.MSELoss()(pred, label.to(device))
        total_loss += (loss.item() / len(loader))

print(f'total_loss : {total_loss:.4f}')

total_loss : 0.5469
